In [ ]:
import torch, numpy, torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms
import torch.optim as optim
import matplotlib.pyplot as plt
from random import randrange
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import wandb
from sklearn.model_selection import KFold
from IPython.display import IFrame
from src.dataset import Dataset
from src.modelling import DeepModel_Trainer
from models.UNet import UNet
from torcheval.metrics.functional import multiclass_f1_score
import numpy as np
import seaborn as sns
import pandas as pd

In [ ]:
wandb.login(key = '8692bab115d599f0d9ca9468f0ea850beb5131b7')

In [ ]:
Unet=UNet()

In [ ]:
test = DeepModel_Trainer('../AgroLuege/raw_data/ZueriCrop/ZueriCrop.hdf5', 'labels.csv', Unet, 'cpu')

In [ ]:
test.train_model('test_plot', 'Unet', 'all', num_epochs=1, test_model=True)

In [ ]:
wandb.finish()

In [ ]:
test.target_local_1


In [ ]:
test.preds0[0][0]

In [ ]:
test.preds1[0][0]

In [ ]:
loss_module = nn.CrossEntropyLoss(weight=torch.tensor([10**-40, 1, 1, 1, 1, 1]))
tt = loss_module(test.preds1, test.target_local_1)

In [ ]:
tt